In [ ]:
# импортируем необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# загружаем датасет
file_path = '/kaggle/input/titanic2/Titanic.csv'
titanic_data = pd.read_csv(file_path)

titanic_data.head()

In [ ]:
# проверим пропущенные значения
missing_data = titanic_data.isnull().sum()
missing_data[missing_data > 0].sort_values(ascending=False)

In [ ]:
# визуализируем выживаемость в зависимости от пола
plt.figure(figsize=(8,6))
sns.countplot(x='Sex', hue='Survived', data=titanic_data)
plt.title('выживаемость в зависимости от пола')
plt.show()

In [ ]:
# построим график выживаемости в зависимости от класса каютв
plt.figure(figsize=(8,6))
sns.countplot(x='Pclass', hue='Survived', data=titanic_data)
plt.title('выживаемость от класса')
plt.show()

In [ ]:
# исследуем распределение возраста и выживаемость
plt.figure(figsize=(10,6))
sns.histplot(titanic_data[titanic_data['Survived'] == 1]['Age'], bins=20, color='green', label='Survived', kde=True)
sns.histplot(titanic_data[titanic_data['Survived'] == 0]['Age'], bins=20, color='red', label='Not Survived', kde=True)
plt.title('распределение выживших и не выживших в зависимости от возраста')
plt.legend()
plt.show()

In [ ]:
# проверим, влиял ли порт посадки на выживаемость
plt.figure(figsize=(8,6))
sns.countplot(x='Embarked', hue='Survived', data=titanic_data)
plt.title('выживаемость по порту посадки')
plt.show()

In [ ]:
# извлечем обращения из имен
titanic_data['Title'] = titanic_data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
# распределим данные о возрасте и стоимости по категорияи
titanic_data['AgeBin'] = pd.cut(titanic_data['Age'], bins=[0, 12, 18, 35, 60, 80], labels=['Child', 'Teenager', 'Adult', 'Middle Aged', 'Senior'])
titanic_data['FareBin'] = pd.qcut(titanic_data['Fare'], 4, labels=['Low', 'Medium', 'High', 'Very High'])
# построим графики влияния получившихся категорий на выживаемость
plt.figure(figsize=(8,6))
sns.countplot(x='AgeBin', hue='Survived', data=titanic_data)
plt.title('Survival by Age Bin')
plt.show()

plt.figure(figsize=(8,6))
sns.countplot(x='FareBin', hue='Survived', data=titanic_data)
plt.title('Survival by Fare Bin')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# заполним пустые значения
titanic_data['Age'].fillna(titanic_data['Age'].median(), inplace=True)
titanic_data['Fare'].fillna(titanic_data['Fare'].median(), inplace=True)
titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0], inplace=True)

# с помощью LabelEncoder преобразуем категориальные значения
label_encoder = LabelEncoder()
titanic_data['Sex'] = label_encoder.fit_transform(titanic_data['Sex'])
titanic_data['Embarked'] = label_encoder.fit_transform(titanic_data['Embarked'])
titanic_data['Title'] = label_encoder.fit_transform(titanic_data['Title'])
titanic_data['AgeBin'] = label_encoder.fit_transform(titanic_data['AgeBin'])
titanic_data['FareBin'] = label_encoder.fit_transform(titanic_data['FareBin'])

# выберем целевую переменную
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Title', 'AgeBin', 'FareBin']
X = titanic_data[features]
y = titanic_data['Survived']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
model = Sequential()
# добавим слои
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# обучим модель
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc * 100:.2f}%')